In [ ]:
import math
import pandas as pd
import numpy as np
import pickle

import torch
from torch import Tensor   
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_add_pool
from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, MessagePassing
from torch.nn import Linear, CrossEntropyLoss, Sequential, ReLU, BCELoss 
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool as gap,  global_max_pool as gmp, global_add_pool as gsp
import torch.nn as nn

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')

from rdkit import Chem                      
from rdkit.Chem import GetAdjacencyMatrix       
from scipy.sparse import coo_matrix
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs


import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.metrics import classification_report, roc_auc_score

from random import shuffle

import re
import gensim
from torchtext.vocab import build_vocab_from_iterator

from torch.nn.utils.rnn import pad_sequence

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="torch_geometric")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.Generator(device=device)

torch.manual_seed(8) 

import gc

In [ ]:
def onek_encoding_unk(value, choices):
    """
    Creates a one-hot encoding with an extra category for uncommon values.

    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is -1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding

In [ ]:
def atom_features(atom,
                  bool_id_feat=False,
                  explicit_H=False,
                  use_chirality=True):
    if bool_id_feat:
        return [atom_to_id(atom)]
    else:
        results = onek_encoding_unk(
          atom.GetSymbol(),
          [
            'B',
            'C',
            'N',
            'O',
            'F',
            'Si',
            'P',
            'S',
            'Cl',
            'As',
            'Se',
            'Br',
            'Te',
            'I',
            'At',
            'other'
          ]) + onek_encoding_unk(atom.GetDegree(),
                                 [0, 1, 2, 3, 4, 5]) + \
                  [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + \
                  onek_encoding_unk(atom.GetHybridization(), [
                    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                        SP3D, Chem.rdchem.HybridizationType.SP3D2,'other'
                  ]) + [atom.GetIsAromatic()]
        # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
        if not explicit_H:
            results = results + onek_encoding_unk(atom.GetTotalNumHs(),
                                                      [0, 1, 2, 3, 4])
        if use_chirality:
            try:
                results = results + onek_encoding_unk(
                    atom.GetProp('_CIPCode'),
                    ['R', 'S']) + [atom.HasProp('_ChiralityPossible')]
            except:
                results = results + [0, 0, 0
                                     ] + [atom.HasProp('_ChiralityPossible')]

        return results


def bond_features(bond: Chem.rdchem.Bond):
    """
    Builds a feature vector for a bond.

    :param bond: An RDKit bond.
    :return: A list containing the bond features.
    """
    if bond is None:
        fbond = [1] + [0] * (14 - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            (bond.GetIsConjugated() if bt is not None else 0),
            (bond.IsInRing() if bt is not None else 0)
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond

In [ ]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 1024
#a vector representation (1x2048) for molecular feature 

In [ ]:
def morgan_binary_features_generator(mol,
                                     radius: int = MORGAN_RADIUS,
                                     num_bits: int = MORGAN_NUM_BITS):
    """
    Generates a binary Morgan fingerprint for a molecule.
    :param mol: A molecule (i.e., either a SMILES or an RDKit molecule).
    :param radius: Morgan fingerprint radius.
    :param num_bits: Number of bits in Morgan fingerprint.
    :return: A 1D numpy array containing the binary Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features

In [ ]:
def data_process(dataset,batch_size):
    SMILES = dataset['SMILES']
    data_list = []
    
    for smiles in SMILES:
            
        mol = Chem.MolFromSmiles(smiles)     

        #generate a global vector features (binary Morgan fingerprint) and convert them
        mol_feature = torch.tensor(morgan_binary_features_generator(mol))

        xs = []
        for atom in mol.GetAtoms():
            x = atom_features(atom)
            xs.append(x)
            
        x = torch.tensor(xs)
        
        edge_indices, edge_attrs = [], []
        
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
    
            e = bond_features(bond)

            edge_indices += [[i,j],[j,i]]
            edge_attrs += [e, e]
        
        edge_index = torch.tensor(edge_indices)
        edge_index = edge_index.t().to(torch.long).view(2, -1)
        edge_attr = torch.tensor(edge_attrs).view(-1, 14)
        
        y = torch.tensor(int(dataset.loc[dataset['SMILES'] == smiles, 'Activity'].values[0])) #response variable y

        smi = smiles

        # add smiles and num_feature as the attributes
        data = Data(x=x, y=y, edge_index=edge_index, edge_attr = edge_attr, smiles=smi, mol_feature=mol_feature)  
        data_list.append(data)   # store processed data into the list
        
    return DataLoader(data_list, batch_size, shuffle=True,generator=generator)

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [ ]:
def train(epoch,train_loader):
    
    model.train()   
    running_loss = 0 
    correct = 0
    total = 0
    criterion = BCELoss()
    y_val=[]
    y_pred=[]
    for batch in train_loader:

        optimizer.zero_grad()
        outputs = model(batch)
        label = batch.y.view(-1,1)
        loss = criterion(outputs.float(),label.float())
        

        loss.backward()   # Compute the gradient of loss function 
        optimizer.step()  # Update parameters based on gradients.
        running_loss += loss.item()
        # probability that is larger than 0.5, classify as 1 

        pred = (outputs >= 0.5).float()

        total += label.size(0)
        correct += (pred == label).float().sum()
        
        y_val.extend(label.cpu().tolist())
        y_pred.extend(outputs.cpu().tolist())

    auc = roc_auc_score(y_val, y_pred)
    loss = running_loss/len(train_loader)
    accuracy = 100*correct/total
    train_auc.append(auc) 
    train_accuracy.append(accuracy)
    train_loss.append(loss)

In [ ]:
def test(epoch,test_loader):
    model.eval()
    
    running_loss = 0
    correct = 0
    total = 0
    y_val=[]
    y_pred=[]
    
    with torch.no_grad():
        criterion = BCELoss()
        for batch in test_loader:
        
            outputs = model(batch)
            label = batch.y.view(-1,1)

            loss = criterion(outputs.float(), label.float())    
            running_loss += loss.item()
            # probability that is larger than 0.5, classify as 1 
            pred = (outputs >= 0.5).float()

            total += label.size(0)
            correct += (pred == label).float().sum()
            y_val.extend(label.cpu().tolist())
            y_pred.extend(outputs.cpu().tolist())
        auc = roc_auc_score(y_val, y_pred)
        loss = running_loss/len(test_loader)
        accuracy = 100*correct/total
    
        test_accuracy.append(accuracy)
        test_loss.append(loss)
        test_auc.append(auc) 


In [ ]:
#test_set as a whole loader
def test_metrics(test_loader):
    model.eval()

    with torch.no_grad():
        labels = []
        preds = []
        for batch in test_loader:
            
            labels += list(batch.y.view(-1, 1).cpu().numpy())
            preds += list(model(batch).detach().cpu().numpy())
        
        pred_labels = [1 if i > 0.5 else 0 for i in preds]
        auc = roc_auc_score(list(labels), list(preds))
        report = classification_report(labels, pred_labels,output_dict=True)
        return auc, report
    

In [ ]:
class FnGATGCN(torch.nn.Module):
    def __init__(self, num_features_xd=44, n_output=1,output_dim=50, dropout=0.1):
        super(FnGATGCN, self).__init__()

        # graph layers
        self.conv1 = GATConv(num_features_xd, num_features_xd, heads=10)
        self.conv2 = GCNConv(num_features_xd*10, num_features_xd*10)
        self.fc_g1 = torch.nn.Linear(num_features_xd*10*2,1500)
        self.fc_g2 = torch.nn.Linear(1500, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)


        self.linear1 = Linear(50, 10)
       

        self.linear2 = Linear(1024,100)
        self.linear3 = Linear(100, 25)
        self.linear4 = Linear(25,10)
        self.linear5 = Linear(20,  n_output)
    def forward(self, data):
        # graph input feed-forward
        x, edge_index, batch, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature
        x = self.conv1(x.float(), edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        x = self.relu(x)
        x = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
        x = self.fc_g1(self.dropout(x))
        x = self.relu(x)
        x = self.fc_g2(self.dropout(x))
        x = self.relu(x)
        x = self.linear1(self.dropout(x))
        x = self.relu(x)
        
        mol_feature=mol_feature.reshape(data.num_graphs,1024)
        x1 = self.linear2(self.dropout(mol_feature.float()))
        x1=F.relu(x1)
        x1 = self.linear3(self.dropout(x1))
        x1=F.relu(x1)
        x1 = self.linear4(self.dropout(x1))
        x1=F.relu(x1)
        x = torch.cat([x, x1],dim=1)
        out= torch.sigmoid(self.linear5(x))
        return out

In [ ]:
best_param ={}
best_param["roc_epoch"] = 0
best_param["loss_epoch"] = 0
best_param["valid_roc"] = 0
best_param["valid_loss"] = 9e8

train_set = pd.read_csv('train_df.csv')
valid_set = pd.read_csv('valid_df.csv')
train_loader = data_process(train_set,100)
valid_loader = data_process(valid_set,100)
    
model = FnGATGCN().float()
model.apply(reset_weights)
model.cuda()
weight_decay = 3.0
learning_rate = 3.5  
optimizer = torch.optim.Adam(model.parameters(), lr=10**-learning_rate,weight_decay=10**-weight_decay)
    
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
test_auc = [] 
train_auc = [] 
for epoch in range(800):
    train(epoch,train_loader)
    test(epoch,valid_loader)
    valid_roc = test_auc[-1]
    valid_loss = test_loss[-1]
    train_roc = train_auc[-1]
    if valid_roc > best_param["valid_roc"]:
        best_param["roc_epoch"] = epoch
        best_param["valid_roc"] = valid_roc
        if valid_roc > 0.70:
             torch.save(model, str(epoch)+'.pt')             
    if valid_loss < best_param["valid_loss"]:
        best_param["loss_epoch"] = epoch
        best_param["valid_loss"] = valid_loss

    print("EPOCH:\t"+str(epoch)+'\n'\
        +"train_roc"+":"+str(train_roc)+'\n'\
        +"valid_roc"+":"+str(valid_roc)+'\n'\
#         +"train_roc_mean"+":"+str(train_roc_mean)+'\n'\
#         +"valid_roc_mean"+":"+str(valid_roc_mean)+'\n'\
        )
    if (epoch - best_param["roc_epoch"] >18) and (epoch - best_param["loss_epoch"] >28):        
        break
    
model = torch.load(str(best_param["roc_epoch"])+'.pt')
auc, report = test_metrics(valid_loader)

In [ ]:
# evaluate model
model = torch.load(str(best_param["roc_epoch"])+'.pt')     


test_set = pd.read_csv('test_df.csv')
test_loader = data_process(test_set,50)

test_auc, report = test_metrics(test_loader)

print("best epoch:"+str(best_param["roc_epoch"])
      +"\n"+"test_roc:"+str(test_auc)
      +"\n"+"test_roc_mean:",str(np.array(test_auc).mean())
     )